In [1]:
class Package:
    def __init__(self, package_id, package_name,num_of_website, cost_per_month_dollar, storage_gb, disc_cache, disc_a_gb, disc_b_gb, cpu_percent, cpu_tic):
        self.package_id = package_id
        self.package_name = package_name
        self.num_of_website = num_of_website
        self.cost_per_month_dollar = cost_per_month_dollar
        self.storage_gb = storage_gb
        self.disc_cache = disc_cache
        self.disc_a_gb = disc_a_gb
        self.disc_b_gb = disc_b_gb
        self.cpu_percent = cpu_percent
        self.cpu_tic = cpu_tic

# Define the packages
package_types=[]

free_package = Package(1,"Free",1, 0, 0, 0, 0, 0, 0 , 0)
essential_package = Package(2, "Essential",1, 49, 10, 20, 50, 50, 80, 100)
advanced_package = Package(3, "Advanced",3, 99, 25, 50, 100, 100, 100, 150)
expert_package = Package(4, "Expert",25, 199, 50, 100, 200, 200, 100, 200)
package_types.append(free_package)
package_types.append(essential_package)
package_types.append(advanced_package)
package_types.append(expert_package)

In [22]:
#pip install faker
import random
import uuid
import time
import json
import threading
from faker import Faker
faker=Faker()
class generate_fake_data:
    def __init__(self):
        self.users_list=[]
        self.sites_list=[]
        self.metrics_list=[]
        self.fake_user_package_list=[]
        self.users_id=[]
        self.sites_id=[]
        self.user_packages_id=[]
        self.index=0
        self.order_list=[]
        
    def fake_user(self): #1000 users
        user_id = str(uuid.uuid4())
        first_name=faker.first_name()
        last_name=faker.last_name()
        address=faker.address()
        email=faker.email()
        phone_number=faker.phone_number()
        user={}
        data = {
            "user_id": user_id,
            "first_name": first_name,
            "last_name": last_name,
            "address":address,
            "email":email,
            "phone_number":phone_number,
              }
        return data
    
    def fake_user_package(self):#1000 package
        for user in self.users_id:
            package_user_id = str(uuid.uuid4())
            package_id = random.choice(package_types).package_id
            for i in package_types:
                if i.package_id==package_id:
                    package=i
                    break;
            remain_sites = package.num_of_website
            storage_gb = package.storage_gb
            disc_cache =package.disc_cache
            disc_a_gb = package.disc_a_gb
            disc_b_gb =package.disc_b_gb
            cpu_percent =package.cpu_percent
            cpu_tic =package.cpu_tic
            data = {
                "index":self.index,
                "package_user_id":package_user_id,###
                "user_id":user,
                "package_id":package_id,
                "remain_sites":remain_sites,
                "storage_gb_left":storage_gb,
                "disc_cache_left":disc_cache,
                "disc_a_gb_left":disc_a_gb,
                "disc_b_gb_left":disc_b_gb,
                "cpu_percent_left":cpu_percent,
                "cpu_tic_left":cpu_tic
                    }
            self.fake_user_package_list.append(data)
            self.user_packages_id.append(package_user_id)
            self.index+=1
    
    def fake_single_site(self,user_id, package_id,format):
        site_id = str(uuid.uuid4())
        url = faker.url()
        data = {
            #name site
            "site_id": site_id,
            "url": url,
            "user_id": user_id,
            "storage_gb_wasted":0,
            "disc_cache_wasted":0,
            "disc_a_gb_wasted":0,
            "disc_b_gb_wasted":0,
            "cpu_percent_wasted":0,
            "cpu_tic_wasted":0
        }
        self.generate_fake_order(data,'json')
        
        return data 
        
    def fake_site(self,format): #5000 sites
        for user_package in self.fake_user_package_list:
            num_of_posibilities=int(user_package['remain_sites'])
            num_of_sites= random.randint(0,num_of_posibilities)
            sites_list=[]
            for i in range(num_of_sites):
                data=self.fake_single_site(user_package['user_id'], user_package['package_id'],format)
                self.sites_list.append(data)
                user_package['remain_sites']-=1
                
    def generate_fake_sites_ahead(self, format):
        current_time = int(time.time())
        one_hour_from_now = current_time + 60 * 60  # One hour from now
        half_year_from_now = current_time + int(182.5 * 24 * 60 * 60)  # Half a year ahead
        while one_hour_from_now <= half_year_from_now:
            for user_package in self.fake_user_package_list:
                if user_package['remain_sites'] > 0:
                    site_data = self.fake_single_site(user_package['user_id'], user_package['package_id'], format, one_hour_from_now)
                    self.sites_list.append(site_data)
                    user_package['remain_sites'] -= 1

            one_hour_from_now += 60 * 60  # Increment time by 1 hour
            
    def update_site_data(self,site_id,field,addition):
        for site in self.sites_list:
            if site['site_id']==site_id:
                site_to_update=site
                break;
        site_to_update[field]+=addition
        
    def fake_metrics(self, site):
        event_id = str(uuid.uuid4())
        site_id = site['site_id']
        # Find the package associated with the site owner
        for i in self.fake_user_package_list:
            if i['user_id'] == site['user_id']:
                user_package = i
                break
        metrics = {}
        if random.random() < 0.5:
            if user_package['storage_gb_left']>0:
                wasted_this_time= random.randint(0, user_package['storage_gb_left']) 
                self.fake_user_package_list[user_package['index']]['storage_gb_left'] -= wasted_this_time 
                self.update_site_data(site_id,'storage_gb_wasted',wasted_this_time)
                metrics["storage_gb"] = site['storage_gb_wasted']
        if random.random() < 0.5:
            if user_package['disc_cache_left']>0:
                wasted_this_time= random.randint(0, user_package['disc_cache_left']) 
                self.fake_user_package_list[user_package['index']]['disc_cache_left'] -= wasted_this_time 
                self.update_site_data(site_id,'disc_cache_wasted',wasted_this_time)
                metrics["disc_cache"] = site['disc_cache_wasted']
        if random.random() < 0.5:
            if user_package['disc_a_gb_left']>0:
                wasted_this_time= random.randint(0, user_package['disc_a_gb_left']) 
                self.fake_user_package_list[user_package['index']]['disc_a_gb_left'] -= wasted_this_time 
                self.update_site_data(site_id,'disc_a_gb_wasted',wasted_this_time)
                metrics["disc_a_gb"] = site['disc_a_gb_wasted']
        if random.random() < 0.5:
            if user_package['disc_b_gb_left']>0:
                wasted_this_time= random.randint(0, user_package['disc_b_gb_left']) 
                self.fake_user_package_list[user_package['index']]['disc_b_gb_left'] -= wasted_this_time 
                self.update_site_data(site_id,'disc_b_gb_wasted',wasted_this_time)
                metrics["disc_b_gb"] = site['disc_b_gb_wasted']
        if random.random() < 0.5:
            if user_package['cpu_percent_left']>0:
                wasted_this_time= random.randint(0, user_package['cpu_percent_left']) 
                self.fake_user_package_list[user_package['index']]['cpu_percent_left'] -= wasted_this_time 
                self.update_site_data(site_id,'cpu_percent_wasted',wasted_this_time)
                metrics["cpu_percent"] = site['cpu_percent_wasted']
        if random.random() < 0.5:
            if user_package['cpu_tic_left']>0:
                wasted_this_time= random.randint(0, user_package['cpu_tic_left']) 
                self.fake_user_package_list[user_package['index']]['cpu_tic_left'] -= wasted_this_time 
                self.update_site_data(site_id,'cpu_tic_wasted',wasted_this_time)
                metrics["cpu_tic"] = site['cpu_tic_wasted']
        data = {
            "event_uuid": event_id,
            "identifier": {
                "site_id": site_id
            },
            "metrics": metrics
        }
        return data
    def generate_fake_users(self):#1000 users
        for _ in range(100):
            fake_user = self.fake_user()
            self.users_list.append(fake_user)
            self.users_id.append(fake_user['user_id'])

    def fake_metrics_for_half_year(self):
        start_time = int(time.time()) - int(182.5 * 24 * 60 * 60)  # Half a year in seconds
        for i in range(182):
            for site in self.sites_list:
                fake_metrics = self.fake_metrics(site)
                start_time += 20
                fake_metrics["event_time"] = start_time
                self.metrics_list.append(fake_metrics)
            start_time += 60 * 60 * 24  # Increment time by 1 day

    def generate_fake_metrics_ahead(self, format):
        while True:
            for site in self.sites_list:
                fake_metrics = self.fake_metrics(site)
                self.metrics_list.append(fake_metrics)
                metrics_file = f'fake_metrics.{format}'
                with open(metrics_file, "w") as file:
                    json.dump(self.metrics_list, file)
                print(f"Fake metrics have been exported to site {site['site_id']}")
            time.sleep(100)  # Sleep for 1 day

    
    def fake_metrics_for_half_year_ahead(self, format):
        threading.Thread(target=self.generate_fake_metrics_ahead, args=(format,)).start()
        metrics_file = f'fake_metrics.{format}'
        with open(metrics_file, "w") as file:
            json.dump(self.metrics_list, file)
        print(f"!Fake metrics have been exported to {metrics_file}")
    
    def fake_sites_ahead(self, format):
        threading.Thread(target=self.generate_fake_sites_ahead, args=(format,)).start()
        print("Fake sites generation has been started.")        
        
    def init_fake_data(self,format):
        self.generate_fake_users()
        users_file = f'fake_users.{format}'
        with open(users_file, "w") as file:
            json.dump(self.users_list, file)
        print(f"Fake users has been exported to {users_file}")
        
        self.fake_user_package()
        user_package = f'fake_user_package.{format}'
        with open(user_package, "w") as file:
            json.dump(self.fake_user_package_list, file)
        print(f"Fake user to package has been exported to {user_package}")
        
        self.fake_site(format)
        sites_file = f'fake_sites.{format}'
        with open(sites_file, "w") as file:
            json.dump(self.sites_list, file)
        print(f"Fake sites has been exported to {sites_file}")
        
        self.fake_metrics_for_half_year()
        metrics_file = f'fake_metrics.{format}'
        with open(metrics_file, "w") as file:
             json.dump(self.metrics_list, file)
        print(len(self.metrics_list))
        
        print(f"Fake metrics has been exported to {metrics_file}")
        sites_file = f'fake_sites.{format}'
        with open(sites_file, "w") as file:
            json.dump(self.sites_list, file)
        print(f"Updated fake sites has been exported to {sites_file}")
        
        user_package = f'fake_user_package.{format}'
        with open(user_package, "w") as file:
            json.dump(self.fake_user_package_list, file)
        print(f"Fake user to package has been exported to {user_package}")
    
    def generate_fake_order(self,site,format):
        for user in self.users_list:
            if user['user_id']==site['user_id']:
                user_name=user['first_name']+' '+user['last_name']
        for package_user in self.fake_user_package_list:
            if package_user['user_id']==site['user_id']:
                for package_type in package_types:
                    if package_user['package_id']==package_user['package_id']:
                        package_name=package_type.package_name
        event_uuid=str(uuid.uuid4())
        event_time = int(time.time())
        user_id=site['user_id']
        user_name=user_name
        site_id=site['site_id']
        package=package_name
        data={
            'event_id':event_uuid,
            'event_time':event_time,
            'user_id':user_id,
            'user_name':user_name,
            'site_id':site_id,
            'package':package  
        }
        self.order_list.append(data)
        order_file = f'fake_order.{format}'
        with open(order_file, "w") as file:
             json.dump(self.order_list, file)
        
        
fake_data_generator = generate_fake_data()

Fake metrics have been exported to site 8352e88e-f3a0-4840-a9ee-7bd186ff4d67
Fake metrics have been exported to site 7df08501-abad-43ad-84ac-b9f3ed6aab22


In [23]:
fake_data_generator.init_fake_data('json')

Fake users has been exported to fake_users.json
Fake user to package has been exported to fake_user_package.json
Fake metrics have been exported to site 403a970e-5ba1-4237-b505-1671241ae522
Fake metrics have been exported to site f4076f8a-89e6-4b0e-af0d-c66dcb3c06a5
Fake sites has been exported to fake_sites.json
Fake metrics have been exported to site 3669df7d-97ea-4000-b5b8-dca563ad3efb
67704
Fake metrics has been exported to fake_metrics.json
Updated fake sites has been exported to fake_sites.json
Fake user to package has been exported to fake_user_package.json


In [24]:
fake_data_generator.fake_metrics_for_half_year_ahead('json')
fake_data_generator.fake_sites_ahead('json')

Fake metrics have been exported to site 438c0cae-f5d4-4664-ba4f-645561ceb281
!Fake metrics have been exported to fake_metrics.json
Fake sites generation has been started.


Exception in thread Thread-11 (generate_fake_sites_ahead):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\The user\AppData\Local\Temp\ipykernel_25132\3726934635.py", line 107, in generate_fake_sites_ahead
TypeError: generate_fake_data.fake_single_site() takes 4 positional arguments but 5 were given


Fake metrics have been exported to site 8b3cbc5d-ac8a-4c44-93a5-8330c2bb8b8a
Fake metrics have been exported to site 7dfd6d43-0a43-4f90-8abe-4d20611adb2d
Fake metrics have been exported to site 06e67628-0d21-4e14-b890-0fc66b1f1c18
Fake metrics have been exported to site c30d4761-e486-44af-99f8-f1555c731cb7
Fake metrics have been exported to site 58ad97cf-ea4d-496f-a005-5a5b8a106bb5
